In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def choose(p):
    a = max(np.flatnonzero(np.append(-np.spacing(1), np.cumsum(p)) < np.random.uniform()))
    #print(a)
    return a

In [ ]:
def simulate_M1random_v1(T, mu, b):
    
    # preallocate arrays pythonically ('a' ~ choice selection; 'r' ~ reward)
    a = [0]*T
    r = [0]*T
    
    for t in range(T):
        
        # compute choice probabilities
        p = np.array([b, 1-b])
        
        # make choice according to choice probabilities
        #a.append(choose(p));
        a[t] = choose(p);
       
        # generate reward based on choice
        #r.append(np.random.uniform() < mu[a])
        r[t] = np.random.uniform() < mu[a[t]]
        
    return a, r

In [6]:
def simulate_M2WSLS_v1(T, mu, epsilon):
    
    a = [0]*T
    r = [0]*T
    
    # initialize last reward/action with nan
    rLast = np.nan
    aLast = np.nan
    
    for t in range(T):
        
        # compute choice probabilites
        if np.isnan(rLast):
            
            # first trial choose randomly
            p = np.array([0.5, 0.5])
            
        else:
            
            # choice depends on last reward
            if rLast == 1:
                
                # win stay (with probability 1-epsilon)
                p = epsilon/2 * np.array([1, 1])
                p[aLast] = 1 - epsilon / 2
                
            else:
                
                # lost shift (with probability 1-epsilon)
                p = (1-epsilon/2) * np.array([1, 1])
                p[aLast] = epsilon / 2
                
        # make choice according to choice probabilities
        a[t] = choose(p)
        
        # generate reward based on choice
        r[t] = np.random.uniform() < mu[a[t]]
        
        aLast = a[t]
        rLast = r[t]
        
    return a, r
        

In [ ]:
def simulate_M3RescorlaWagner_v1(T, mu, alpha, beta):
    
    a = [0]*T
    r = [0]*T
    
    Q = np.array([0.5, 0.5])
    
    for t in range(T):
        
        # compute choice probabilities
        p = np.exp(beta * Q) / np.sum(np.exp(beta * Q))
        #p = math.exp()
                                               
        # make choice according to choice probabilities
        a[t] = choose(p)
        
        # generate reward based on choice
        r[t] = np.random.uniform() < mu[a[t]]
        
        # update values
        delta = r[t] - Q[a[t]]
        Q[a[t]] = Q[a[t]] + alpha * delta
        
    return a, r
        

In [ ]:
def simulate_M4ChoiceKernel_v1(T, mu, alpha_c, beta_c):
    
    a = [0]*T
    r = [0]*T
    
    CK = np.array([0, 0])
    
    for t in range(T):
        
        # compute choice probabilities
        p = np.exp(beta_c * CK) / np.sum(np.exp(beta_c * CK))
        
        # make choice according to choice probabilities
        a[t] = choose(p)
        
        # generate reward based on choice
        r[t] = np.random.uniform() < mu[a[t]]
        
        # update choice kernel
        CK = (1-alpha_c) * CK
        CK[a[t]] = CK[a[t]] + alpha_c * 1
        
    return a, r


In [ ]:
def simulate_M5RWCK_v1(T, mu, alpha, beta, alpha_c, beta_c):
    
    a = [0]*T
    r = [0]*T
    
    Q = np.array([0.5, 0.5])
    CK = np.array([0, 0])
    
    for t in range(T):
        
        # compute choice probabilities
        V = beta * Q + beta_c * CK
        p = np.exp(V) / np.sum(np.exp(V))
        
        # make choice according to choice probabilities
        a[t] = choose(p)
        
        # generate reward based on choice
        r[t] = np.random.uniform() < mu[a[t]]
        
        # update values
        delta = r[t] - Q[a[t]]
        Q[a[t]] = Q[a[t]] + alpha * delta
        
        # update choice kernel
        CK = (1-alpha_c) * CK
        CK[a[t]] = CK[a[t]] + alpha_c * 1
        
    return a, r